In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
##样本点[-0.5,0.5]
x_data=np.linspace(-0.5,0.5,200)[:,np.newaxis]
noise=np.random.normal(0,0.02,x_data.shape)
y_data=np.square(x_data)+noise


##placeholder

x=tf.placeholder(tf.float32,[None,1])
y=tf.placeholder(tf.float32,[None,1])

###神经网络中间层
weights_L1=tf.Variable(tf.random_normal([1,10]))
biases_L1=tf.Variable(tf.zeros([1,10]))
wx_plus_b_L1=tf.matmul(x,weights_L1) + biases_L1
L1=tf.nn.tanh(wx_plus_b_L1)


###神经网络输出层
weights_L2=tf.Variable(tf.random_normal([10,1]))
biases_L2=tf.Variable(tf.zeros([1,1]))
wx_plus_b_L2=tf.matmul(L1,weights_L2) + biases_L2
prediction=tf.nn.tanh(wx_plus_b_L2)


##二次代价函数
loss=tf.reduce_mean(tf.square(y-prediction))

#梯度下降
train_step=tf.train.GradientDescentOptimizer(0.1).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(2000):
        sess.run(train_step, feed_dict={x:x_data, y:y_data})
    
    prediction_value=sess.run(prediction, feed_dict={x:x_data})
    
    ##画图
    plt.figure()
    plt.scatter(x_data, y_data)
    plt.plot(x_data, prediction_value, 'r-', lw=5)
    plt.show()

In [ ]:
mnist=input_data.read_data_sets("MNIST_data", one_hot=True)

In [2]:
mnist=input_data.read_data_sets("MNIST_data", one_hot=True)
batch_size=128
n_batch=mnist.train.num_examples // batch_size

with tf.name_scope('input'):
##定义placeholder
    x=tf.placeholder(tf.float32,[None,784], name='x-input')
    y=tf.placeholder(tf.float32,[None,10], name='y-input')
    keep_prob=tf.placeholder(tf.float32, name='dropout')

with tf.name_scope('layer'):
###神经网络中间层
    weights_L1=tf.Variable(tf.truncated_normal([784,500],stddev=0.1))
    biases_L1=tf.Variable(tf.zeros([500])+0.1)
    wx_plus_b_L1=tf.matmul(x,weights_L1) + biases_L1
    L1=tf.nn.relu(wx_plus_b_L1)
    L1_drop=tf.nn.dropout(L1,keep_prob)
    
    # weights_L2=tf.Variable(tf.zeros([500,300]))
    # biases_L2=tf.Variable(tf.zeros([300]))
    # wx_plus_b_L2=tf.matmul(L1_drop,weights_L2) + biases_L2
    # L2=tf.nn.relu(wx_plus_b_L2)
    
    # ###神经网络输出层
    weights_L3=tf.Variable(tf.truncated_normal([500,10],stddev=0.1))
    biases_L3=tf.Variable(tf.zeros([10])+0.1)
    wx_plus_b_L3=tf.matmul(L1_drop,weights_L3) + biases_L3
    prediction=tf.nn.softmax(wx_plus_b_L3)

with tf.name_scope('loss'):
##二次代价函数
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    tf.summary.scalar('loss', loss)
    
with tf.name_scope('train_step'):
    train_step=tf.train.AdamOptimizer(1e-3).minimize(loss)

init = tf.global_variables_initializer()

with tf.name_scope('correct_prediction'):
    correct_prediction=tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))

with tf.name_scope('accuarcy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

merged=tf.summary.merge_all()
    
with tf.Session() as sess:
    sess.run(init)
    writer=tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys= mnist.train.next_batch(batch_size)
            summary, _ = sess.run([merged,train_step], feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.5})
            
        writer.add_summary(summary, epoch)    
        acc=sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels,keep_prob:1})
        print("iter" + str(epoch)+", testing accuracy:" + str(acc))
    

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input

In [ ]:
plt.figure()
plt.plot(acc, 'r-', lw=5)
plt.show()